# Proyecto Final 

### Asignatura: 
        Portafolios de Inversión 

### Integrantes: 
        1) Iván Andrés Arellano (714383)
        2) Fernando Aguilar Carvajal (IF724031)
        3) xxxx_xx


In [35]:
# Se importan librerías 
import matplotlib
import numpy as np
import pandas as pd
import scipy.stats as stats
import plotly.express as px
import scipy.optimize as opt
import plotly.graph_objs as go
import matplotlib.pyplot as plt
from scipy.optimize import minimize
import pandas_datareader.data as web
from plotly.subplots import make_subplots

In [36]:
# Funciones: 

# Función para descargar precios de cierre ajustados:
def get_adj_closes(tickers, start_date=None, end_date=None):
    # Fecha inicio por defecto (start_date='2010-01-01') y fecha fin por defecto (end_date=today)
    # Descargamos DataFrame con todos los datos
    closes = web.DataReader(name=tickers, data_source='yahoo', start=start_date, end=end_date)
    # Solo necesitamos los precios ajustados en el cierre
    closes = closes['Adj Close']
    # Se ordenan los índices de manera ascendente
    closes.sort_index(inplace=True)
    return closes

# Función Varianza
def varianza(w,Sigma):
    return w.T.dot(Sigma).dot(w) 

# Función objetivo
def menos_RS(w,Eind,Sigma,rf):
    Ep = Eind.T.dot(w)
    sp = (w.T.dot(Sigma).dot(w))**.5
    RS = (Ep-rf)/sp
    return -RS

# Función Cambio Porcentual
def pct_chng(VA: 'Valor Actual',
             VI: 'Valor Inicial'):
    
    ret = np.round(((VA-VI)/VI), 2)
    return ret


def simulacion_Portafolio(ret: 'Retornos',
                       rf: 'Tasa Libre de Riesgo',
                       g: 'Grado de Aversión al Riesgo'):
    
    global pesos, summary, w_EMV, w_opt, E_EMV, s_EMV,tickers
        
    #Rendimientos y volatilidades esperadas
    summary = pd.DataFrame(data={'Media': 252*ret.mean(),
                                 'Vol':((252)**(.5))*ret.std()})
        
    cov_ret = ret.cov() # Matriz de varianza - covarianza
    
    corr = ret.corr() # Matriz de correlación
    
    S = np.diag(summary['Vol'].values)
    Sigma = S.dot(corr).dot(S)
    Eind = summary['Media'].values
    
    n = len(Eind) # Número de activos    
    w0 = np.ones((n,))/n # Dato inicial   
    bnds = ((0, 1),)*n # Cotas de las variables    
    cons={'type':'eq','fun':lambda w:w.sum()-1} # Restricciones
    
    # Portafolio de mínima varianza
    minvar = minimize(fun=varianza,
                      x0=w0,
                      args=(Sigma,),
                      bounds=bnds,
                      constraints=cons,
                      tol=1e-5)
    
    w_minvar = minvar.x
    
    E_minvar = Eind.T.dot(w_minvar)
    s_minvar = (w_minvar.T.dot(Sigma).dot(w_minvar))**.5
    RS_minvar = (E_minvar - rf)/s_minvar

    # Portafolio EMV
    EMV = minimize(fun = menos_RS,
                   x0 = w0,
                   args = (Eind, Sigma, rf),
                   bounds = bnds,
                   constraints = cons,
                   tol = 1e-5)
    w_EMV = EMV.x
    
    # Pesos, rendimiento y riesgo del portafolio EMV
    E_EMV = Eind.T.dot(w_EMV)
    s_EMV = (w_EMV.T.dot(Sigma).dot(w_EMV))**0.5
    RS_EMV = (E_EMV - rf)/s_EMV
    
    #Asignación de capital
    w_opt = (E_EMV - rf)/(g*s_EMV**2)
    
    pesos = [w_EMV[i]*w_opt for i in range(len(w_EMV))]
    pesos.append(1-w_opt)
    pesos = np.round(pesos, 4)
    
    tickers = ['XLK.MX','VOO.MX','VGT.MX','VHT.MX',
              'VOX.MX','BBVA.MX','AC.MX','AMXL.MX',
              'BIMBOA.MX','GAPB.MX','C.MX',
              'AEROMEX.MX','WALMEX.MX','SAN.MX',
                'GMEXICOB.MX', 'Cetes']
    
    a_c = pd.DataFrame(columns = ['Asignación de Capital'], index=[tickers])
    a_c['Asignación de Capital'] = pesos
    
    return a_c



In [28]:
ETFS = ['XLK.MX','VOO.MX','VGT.MX','VHT.MX','VOX.MX']

Stoks = ['BBVA.MX','AC.MX','AMXL.MX',
         'BIMBOA.MX','CUERVO.MX','VOLARA.MX',
         'AEROMEX.MX','WALMEX.MX','SAN.MX','GMEXICOB.MX']

tickers = ['XLK.MX','VOO.MX','VGT.MX','VHT.MX',
           'VOX.MX','BBVA.MX','AC.MX','AMXL.MX',
           'BIMBOA.MX','GAPB.MX','C.MX',
           'AEROMEX.MX','WALMEX.MX','SAN.MX','GMEXICOB.MX' ]

## Datos Historicos previos al 2015-12-31

In [33]:
start_date = '2011-12-31'
end_date = '2016-12-31'
closes_prev = get_adj_closes(tickers=tickers,
                        start_date= start_date , end_date=end_date)
ret_prev = closes_prev.pct_change().dropna()
Tasa_Fija2012_5y = 0.0546
g = 7

In [47]:
Port_prev = simulacion_Portafolio(ret_prev,Tasa_Fija2012_5y,g)
W_port1 = pesos
Port_prev

,Asignación de Capital
XLK.MX,0.0119
VOO.MX,0.0127
VGT.MX,0.0091
VHT.MX,0.0071
VOX.MX,0.0083
BBVA.MX,0.0000
AC.MX,0.0000
AMXL.MX,0.0000
BIMBOA.MX,0.0000
GAPB.MX,0.7802


## 2016 

In [13]:
# Portafolio 2016
start_date = '2015-12-31'
end_date = '2016-12-31'
closes_16 = get_adj_closes(tickers=tickers,
                        start_date= start_date , end_date=end_date)

In [14]:
closes_16.tail()

Symbols,XLK.MX,VOO.MX,VGT.MX,VHT.MX,VOX.MX,BBVA.MX,AC.MX,AMXL.MX,BIMBOA.MX,GAPB.MX,VOLARA.MX,AEROMEX.MX,WALMEX.MX,SAN.MX,GMEXICOB.MX
Date,,,,,,,,,,,,,,,
2016-12-26,1009.52002,NaN,NaN,NaN,NaN,138.163254,91.983765,11.295855,41.995289,130.713608,29.770000,39.209999,32.701836,49.488335,44.403854
2016-12-27,1025.00000,4317.299805,123.760002,2657.899902,101.400002,138.163254,92.139725,11.330746,41.657742,130.596573,30.040001,39.150002,32.640266,50.095276,44.523273
2016-12-28,1020.00000,4284.430176,122.589996,127.029999,100.610001,136.182419,93.352531,11.339468,42.351570,130.822861,30.299999,39.250000,32.860168,48.802040,45.972366
2016-12-29,1004.00000,4220.000000,2534.840088,2633.270020,100.779999,136.281479,94.435402,11.452862,44.011127,132.328735,31.040001,39.230000,33.027275,48.993462,45.948467
2016-12-30,1010.00000,4252.000000,2534.840088,126.650002,100.000000,137.668060,93.621078,11.365636,44.076763,133.015335,31.010000,39.099998,32.587494,49.815151,44.929348


In [20]:
ret_16 = closes_16.pct_change().dropna()